<a href="https://colab.research.google.com/github/dahlia52/Advanced-Statistical-Data-Analysis/blob/main/seq_cell_classification_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.transforms as transforms
from torchvision.datasets import MNIST, CIFAR10, CIFAR100
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt

In [2]:
path = './datasets/'

transform = transforms.Compose([transforms.ToTensor()]) # 이미지를 텐서로 변환

# Prepare Data
train_data = MNIST(root = path, train = True, transform = transform, download = True)
test_data = MNIST(root = path, train = False, transform = transform, download = True)

batch_size = 100

# DataLoader
train_loader = DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True, num_workers = 4)
test_loader = DataLoader(dataset = test_data, batch_size = batch_size, shuffle = False, num_workers = 4)

print(train_data)
print(test_data)

100%|██████████| 9912422/9912422 [00:00<00:00, 53104617.49it/s]


Extracting ./datasets/MNIST/raw/train-images-idx3-ubyte.gz to ./datasets/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 114278956.44it/s]


Extracting ./datasets/MNIST/raw/train-labels-idx1-ubyte.gz to ./datasets/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 46420674.82it/s]

Extracting ./datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to ./datasets/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 16268598.44it/s]


Extracting ./datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./datasets/MNIST/raw

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./datasets/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )
Dataset MNIST
    Number of datapoints: 10000
    Root location: ./datasets/
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
           )


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [3]:
if torch.backends.mps.is_available():
    device = torch.device("mps:0")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu:0")

print(device)

cuda:0


In [4]:
_, seq_len, input_size = train_data[0][0].shape # (1,28,28)
output_shape = len(train_data.classes) # 10
hidden_size = input_size * 2

model_name = "rnn"

MNIST 데이터를 RNN에 적용

- row 별 28개의 값을 하나의 데이터 인스턴스로 sequential하게 넣음.

In [5]:
class RNNClassifier(nn.Module):

  def __init__(self, input_size, hidden_size):
    super().__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size

    self.cell = nn.RNNCell(input_size = self.input_size, hidden_size = self.hidden_size)
    self.fc = nn.Linear(self.hidden_size, output_shape)

  def forward(self, x):
    x = x.reshape(-1, seq_len, self.input_size).permute((1,0,2)) # (100,1,28,28) -> (100,28,28) -> (28,100,28) => for루프를 위해 transpose
    hidden_state = torch.zeros(batch_size, self.hidden_size).to(device) # initial hidden value 설정 (like 수열의 첫 번째 값 설정)
    for i in range(seq_len): # 100, 28 = batch_size, input_size
       hidden_state = self.cell(x[i], hidden_state) # ith input과 past hidden state를 이용하여 current hidden state update
    out = self.fc(hidden_state)

    return out

In [6]:
class LSTMClassifier(nn.Module):

  def __init__(self, input_size, hidden_size):
    super().__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size

    self.cell = nn.LSTMCell(input_size = self.input_size, hidden_size = self.hidden_size)
    self.fc = nn.Linear(self.hidden_size, output_shape)

  def forward(self, x):
    x = x.reshape(-1, seq_len, self.input_size).permute((1,0,2))
    hidden_state = torch.zeros(batch_size, self.hidden_size).to(device)
    cell_state = torch.zeros(batch_size, self.hidden_size).to(device) # initial cell state 설정

    for i in range(seq_len):
      hidden_state, cell_state = self.cell(x[i], (hidden_state, cell_state))
    out = self.fc(hidden_state)

    return out

In [7]:
class GRUClassifier(nn.Module):

  def __init__(self, input_size, hidden_size):
    super().__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size

    self.cell = nn.GRUCell(input_size = self.input_size, hidden_size = self.hidden_size)
    self.fc = nn.Linear(self.hidden_size, output_shape)

  def forward(self, x):
    x = x.reshape(-1, seq_len, input_size).permute((1,0,2))
    hidden_state = torch.zeros(batch_size, self.hidden_size).to(device)
    for i in range(seq_len):
      hidden_state = self.cell(x[i], self.hidden_state).to(device)
    out = self.fc(hidden_state)

    return out

In [8]:
device = torch.device("cuda:0")

if model_name == "rnn":
  classifier = RNNClassifier
elif model_name == "lstm":
  classifier = LSTMClassifier
elif model_name == "gru":
  classifier = GRUClassifier

In [9]:
model = classifier(input_size, hidden_size).to(device)
loss = nn.CrossEntropyLoss(reduction = 'sum')

optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

In [15]:
model

RNNClassifier(
  (cell): RNNCell(28, 56)
  (fc): Linear(in_features=56, out_features=10, bias=True)
)

In [11]:
num_epoch = 100
train_loss_list, test_loss_list = list(), list()

for i in range(num_epoch):
  # train
  model.train()
  total_loss = 0
  count = 0

  for batch_idx, (x, y) in enumerate(train_loader):
    x, y = x.to(device), y.to(device)

    y_est = model.forward(x)
    cost = loss(y_est, y)

    total_loss += cost.item()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    pred = torch.argmax(y_est, dim = -1)
    count += (pred == y).sum().item()

  acc = count / len(train_data)
  ave_loss = total_loss / len(train_data)
  train_loss_list.append(ave_loss)

  if i % 1 == 0:
    print("\nEpoch %d Train: Loss %.3f / Accuracy %.3f"%(i,ave_loss,acc))


  # test
  model.eval()

  total_loss = 0
  count = 0

  with torch.no_grad():
    for batch_idx, (x, y) in enumerate(test_loader):
      x, y = x.to(device), y.to(device)
      y_est = model.forward(x)
      cost = loss(y_est, y)

      total_loss += cost.item()

      pred = torch.argmax(y_est, dim = -1)
      count += (pred == y).sum().item()

    acc = count / len(test_data)
    avg_loss = total_loss / len(test_data)

    test_loss_list.append(avg_loss)

    if i % 1 == 0:
      print("Epoch %d Test: Loss %.3f / Accuracy %.3f"%(i,avg_loss,acc))


Epoch 0 Train: Loss 0.893 / Accuracy 0.686
Epoch 0 Test: Loss 0.780 / Accuracy 0.738

Epoch 1 Train: Loss 0.687 / Accuracy 0.768
Epoch 1 Test: Loss 0.603 / Accuracy 0.802

Epoch 2 Train: Loss 0.549 / Accuracy 0.824
Epoch 2 Test: Loss 0.510 / Accuracy 0.835

Epoch 3 Train: Loss 0.440 / Accuracy 0.866
Epoch 3 Test: Loss 0.405 / Accuracy 0.873

Epoch 4 Train: Loss 0.367 / Accuracy 0.892
Epoch 4 Test: Loss 0.311 / Accuracy 0.910

Epoch 5 Train: Loss 0.321 / Accuracy 0.907
Epoch 5 Test: Loss 0.343 / Accuracy 0.895

Epoch 6 Train: Loss 0.286 / Accuracy 0.916
Epoch 6 Test: Loss 0.255 / Accuracy 0.927

Epoch 7 Train: Loss 0.258 / Accuracy 0.927
Epoch 7 Test: Loss 0.246 / Accuracy 0.932

Epoch 8 Train: Loss 0.240 / Accuracy 0.930
Epoch 8 Test: Loss 0.224 / Accuracy 0.941

Epoch 9 Train: Loss 0.216 / Accuracy 0.938
Epoch 9 Test: Loss 0.212 / Accuracy 0.943

Epoch 10 Train: Loss 0.201 / Accuracy 0.943
Epoch 10 Test: Loss 0.207 / Accuracy 0.943

Epoch 11 Train: Loss 0.196 / Accuracy 0.943
Epoch 1

In [13]:
num_parameter = 0
for parameter in model.parameters():
  print(parameter.shape)
  num_parameter += np.prod(parameter.size())
print(num_parameter)

torch.Size([56, 28])
torch.Size([56, 56])
torch.Size([56])
torch.Size([56])
torch.Size([10, 56])
torch.Size([10])
5386
